# Read Overpass-API to Python Pandas Dataframe

The [Overpass API](http://wiki.openstreetmap.org/wiki/Overpass_API) provides access to the data behind the Openstreetmaps Map Data.
The [Overpass-Turbo](http://overpass-turbo.eu/) is the easyiest way to test requests and get the correct code to ask the database. Example: Fountains with wikidata Q-Numbers as per https://github.com/water-fountains/import2wikidata/issues/15

In [1]:
from datetime import datetime as dt
dtFmt = "%y%m%d_%H%M%S"
print (dt.now().strftime(dtFmt))
import pandas as pd
import requests
import json
pd.set_option('display.max_columns', 200)

191124_090453


In [2]:
# Get yours at: http://boundingbox.klokantech.com/
# Zürich ll 47.3211, 8.4217 ur 47.4383, 8.6284
bbox = [8.4217,47.3211,8.6284,47.4383]

# Links unten
minLat = bbox[1]
minLon = bbox[0]

# Rechts oben
maxLat = bbox[3]
maxLon = bbox[2]

### Construct the Overpass Query String

Request from [Overpass-Turbo](http://overpass-turbo.eu/)

In [10]:
bbox_string = '(%s,%s,%s,%s)' % (minLat, minLon, maxLat, maxLon)

compactOverpassQLstring = f'''

[out:json][timeout:60];

(
    node[man_made=drinking_fountain][wikidata]{bbox_string};
    node[amenity=drinking_water][wikidata]{bbox_string};
    node[amenity=fountain][wikidata]{bbox_string};
);'''


#node[man_made=water_tap]{bbox_string};node[natural=spring]{bbox_string};node[amenity=watering_place]{bbox_string};
#node[amenity=water_point]{bbox_string};node[man_made=water_well]{bbox_string};
compactOverpassQLstring += 'out ;'
compactOverpassQLstring

'\n\n[out:json][timeout:60];\n\n(\n    node[man_made=drinking_fountain][wikidata](47.3211,8.4217,47.4383,8.6284);\n    node[amenity=drinking_water][wikidata](47.3211,8.4217,47.4383,8.6284);\n    node[amenity=fountain][wikidata](47.3211,8.4217,47.4383,8.6284);\n);out ;'

In [11]:
osmrequest = {'data': compactOverpassQLstring}
osmurl = 'https://overpass-api.de/api/interpreter'

# Ask the API
osm = requests.get(osmurl, params=osmrequest)

### Reformat the JSON to fit in a Pandas Dataframe

The JSON can't be directyl imported to a Pandas Dataframe:

Thanks to [unutbu from stackoverflow.com](http://stackoverflow.com/questions/24848416/expected-string-or-unicode-when-reading-json-with-pandas) for fiddling this out!

In [12]:
osmdata = osm.json()
# print out the json
osmdata

{'version': 0.6,
 'generator': 'Overpass API 0.7.55.1009 5e627b63',
 'osm3s': {'timestamp_osm_base': '2019-11-24T09:10:02Z',
  'copyright': 'The data included in this document is from www.openstreetmap.org. The data is made available under ODbL.'},
 'elements': [{'type': 'node',
   'id': 60733338,
   'lat': 47.40283,
   'lon': 8.4999404,
   'tags': {'amenity': 'drinking_water',
    'artist_name': 'Hans Jakob Meyer',
    'drinking_water:description': 'Leitungswasser',
    'name': 'Tränkbrunnen Höngg',
    'operator': 'WVZ',
    'ref': '465',
    'wikidata': 'Q27230124'}},
  {'type': 'node',
   'id': 61272331,
   'lat': 47.4092916,
   'lon': 8.4948507,
   'tags': {'amenity': 'drinking_water',
    'artist_name': 'Piero Tedoldi',
    'drinking_water:description': 'Leitungswasser',
    'name': 'Brunnen zur Eingemeidung',
    'operator': 'WVZ',
    'ref': '1091',
    'start_date': '1984',
    'wikidata': 'Q27229864'}},
  {'type': 'node',
   'id': 83325488,
   'lat': 47.3384108,
   'lon': 8.5

In [13]:
osmelements = osmdata['elements']
for dct in osmelements:
    for key, val in dct['tags'].items():
        dct[key] = val
    del dct['tags']

### Now put everything to the Pandas Dataframe

In [14]:
osmdf = pd.DataFrame(osmelements)

### Look at the whole Dataframe

In [16]:
osmdf.head(5)

,type,id,lat,lon,amenity,artist_name,drinking_water:description,name,operator,ref,wikidata,start_date,stateofrepair,wheelchair,access,dog,fee,bottle,source,amenity_1,drinking_water,material,alt_name,official_name,old_name,note,description,uzh_landmark,wheelchair:description,wikimedia_commons,level,fixme,artwork_type,loc_name,tourism,wikipedia,name:ru
0,node,60733338,47.402830,8.499940,drinking_water,Hans Jakob Meyer,Leitungswasser,Tränkbrunnen Höngg,WVZ,465,Q27230124,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,node,61272331,47.409292,8.494851,drinking_water,Piero Tedoldi,Leitungswasser,Brunnen zur Eingemeidung,WVZ,1091,Q27229864,1984,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,node,83325488,47.338411,8.532262,drinking_water,Uli Schoop,Leitungswasser,Brunnentrog,WVZ,158,Q27230215,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,node,83326029,47.338468,8.530644,drinking_water,Armin Wanger,Leitungswasser,Trinkbrunnen Tramendstation Wollishofen,WVZ,164,Q27229839,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,node,83330378,47.344198,8.529788,drinking_water,Paul Vogelsanger;Emil Schäfer,Leitungswasser,"Brunnen mit Relief ""Hund und Katze""",WVZ,144,Q27229951,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [22]:
osmdfShrt =  osmdf[['lat','lon','wikidata']]

In [23]:
osmdfShrt.tail(5)

,lat,lon,wikidata
262,47.382097,8.548770,Q55170211
263,47.360504,8.547794,Q67588104
264,47.360213,8.547904,Q67588045
265,47.360331,8.547823,Q67587947
266,47.365180,8.568826,Q74171778


## Export to CSV

In [24]:
osmCsvFileName = "osmFountainsWithWikidataReference"+dt.now().strftime(dtFmt)+".csv"
osmdfShrt.to_csv(osmCsvFileName, index=False)



In [25]:
print("wrote '"+osmCsvFileName+"' with "+str(len(osmdf))+" lines")

wrote 'osmFountainsWithWikidataReference191124_092526.csv' with 267 lines
